In [2]:
import botocore
import pandas as pd

In [5]:
from sagemaker import get_execution_role
from sagemaker.session import s3_input, Session
from sagemaker.amazon.amazon_estimator import get_image_uri 
from sagemaker.session import s3_input, Session
role = get_execution_role()

In [61]:
data_location = 's3://bankbenefits/bank_clean.csv'
#data_location  = 's3://amazon-reviews-ml/'
model_data = pd.read_csv(data_location)
model_data_test = model_data.head(100)   

In [58]:
new_header = model_data_test.iloc[0] #grab the first row for the header
model_data_test = model_data_test[1:] #take the data less the header row
model_data_test.columns = new_header #set the header row as the df 

In [65]:
model_data_test

,Unnamed: 0,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,0,56,1,999,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
1,1,57,1,999,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
2,2,37,1,999,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
3,3,40,1,999,0,1,0,1,0,0,...,0,1,0,0,0,0,1,0,1,0
4,4,56,1,999,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,45,1,999,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
96,96,42,1,999,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
97,97,53,1,999,0,1,0,1,0,0,...,0,1,0,0,0,0,1,0,1,0
98,98,37,1,999,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [68]:
import pickle
import xgboost as xgb
#pd.concat([model_data_test['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
model_data_array = model_data_test.drop(['y_no', 'y_yes'], axis=1).values
model = pickle.load(open('/home/ec2-user/SageMaker/bankbenefitwithcontainer/model/xgboost-model', 'rb'))
from sagemaker.predictor import csv_serializer
#test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array


#predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
#print(predictions_array.shape)
# prediction with test data
xgtest = xgb.DMatrix(model_data_array )
pred = model.predict(xgtest)

In [69]:
pred

array([0.01756924, 0.01449983, 0.01826371, 0.0171228 , 0.01661756,
       0.01793173, 0.0190145 , 0.01739063, 0.01514495, 0.01686344,
       0.01739063, 0.01686344, 0.02185421, 0.01909351, 0.01988016,
       0.01608681, 0.01988016, 0.02639032, 0.01970534, 0.01803688,
       0.01932164, 0.01749106, 0.01881076, 0.01687693, 0.01826371,
       0.01414503, 0.01784523, 0.01935588, 0.01533226, 0.01431378,
       0.0176125 , 0.01784523, 0.02061168, 0.01700105, 0.01654784,
       0.01541968, 0.01701972, 0.01706688, 0.01952445, 0.01980776,
       0.02217713, 0.02294688, 0.01810534, 0.01840009, 0.02128927,
       0.01531102, 0.02281287, 0.01696762, 0.01984193, 0.02674699,
       0.01661756, 0.0160462 , 0.01701018, 0.01981991, 0.02140696,
       0.01924518, 0.01484263, 0.01991688, 0.01487447, 0.01525418,
       0.02236156, 0.01970534, 0.02695722, 0.02288918, 0.01832005,
       0.01708977, 0.022003  , 0.01846351, 0.02078545, 0.02074696,
       0.01664293, 0.01868886, 0.02128927, 0.01429097, 0.01998

In [41]:
y_pred_nn =np.array(pred)
y_pred_nn2 = (y_pred_nn > 0.025)
y_pred = y_pred_nn2.astype(int)

In [26]:
import sklearn
import sklearn.metrics
import numpy as np
cutoff=0.5
print(sklearn.metrics.confusion_matrix(model_data_test['y_yes'], np.where(y_pred_nn  > cutoff, 1, 0)))
print(sklearn.metrics.classification_report(model_data_test['y_yes'], np.where(y_pred_nn > cutoff, 1, 0)))

[[97  0]
 [ 3  0]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98        97
           1       0.00      0.00      0.00         3

    accuracy                           0.97       100
   macro avg       0.48      0.50      0.49       100
weighted avg       0.94      0.97      0.96       100



/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [48]:
from sklearn.metrics import accuracy_score

y_pred = y_pred_nn2.astype(int)
accuracy_score(model_data_test['y_yes'], y_pred , normalize=False)

93